Так как в приложенных файлах - разные версии ноутбуков с разными заданиями - здесь всё вместе.

In [1]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]
prices_recommended = [80, 200, 400, 235, 150, 70, 25, 50, 100, 30]
prices_bought = [15, 670, 80, 235]

# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [2]:
import numpy as np
import pandas as pd

In [3]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list, bought_list)
    hit_rate = int(flags.sum() > 0) 
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    # your_code
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(k_recommended_list, bought_list)
    hit_rate = int(flags.sum() > 0)
    
    return hit_rate

In [4]:
hit_rate_at_k(recommended_list, bought_list, 1)

1

In [5]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    # your_code
    # Лучше считать через скалярное произведение, а не цикл
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list[:k])
    k_prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(k_recommended_list, bought_list)
    
    precision = (flags * k_prices_recommended).sum() / k_prices_recommended.sum()
    
    return precision

In [6]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, 5)

0.29577464788732394

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [7]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(k_recommended_list, bought_list) 
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list[:k])
    k_prices_recommended = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(k_recommended_list, bought_list)
    precision = (flags * k_prices_recommended).sum() / prices_bought.sum()
    
    return precision

In [8]:
recall_at_k(recommended_list, bought_list, 3)

0.25

In [9]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, 5)

0.315

In [10]:
def ap_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list,bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k):
        
        if flags[i]:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / k
    
    return result

In [11]:
def map_k(recommended_list, bought_list, k=5):
    
    # your_code
    recommended_lists = np.array(recommended_list)
    bought_lists = np.array(bought_list)
    
    result = np.array([ap_at_k(x, y, k) for x, y in zip(recommended_lists, bought_lists)]).mean()
    
    return result

In [12]:
data = pd.DataFrame({"user_id": ["u1","u2","u3"], 
              "recommended_list": [
                                      [143, 156, 1134, 991, 27],
                                      [1543, 3345, 533, 11, 43],
                                      [156, 3345, 10, 15, 1134]
                                  ],
             "bought_list": [
                             [156,27],
                             [11,43],
                             [1]
                            ]})

In [13]:
map_k(data['recommended_list'], data['bought_list'])

0.10333333333333333

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  ReciprocalRank(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [14]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list[:k])

    flags = np.isin(k_recommended_list, bought_list)
    for i, flag in enumerate(flags, start=1):
        if flag:
            return (1/i)
    
    return 0

In [15]:
reciprocal_rank_at_k(recommended_list, bought_list, 5)

1.0

In [16]:
def mean_rrk(recommended_list, bought_list, k=5):
    
    # your_code
    recommended_lists = np.array(recommended_list)
    bought_lists = np.array(bought_list)
    
    result = np.array([reciprocal_rank_at_k(x, y, k) for x, y in zip(recommended_lists, bought_lists)]).mean()
    
    return result

In [17]:
mean_rrk(data['recommended_list'], data['bought_list'])

0.25

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [18]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    k_recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(k_recommended_list, bought_list)
    ideal_dcg = np.array([1/np.log2(i + 2) for i in range(k)])  # +1 начинаем с 0, +1 - по формуле discount
    
    ndcg = (flags * ideal_dcg).sum() / ideal_dcg.sum()
    
    return ndcg

In [19]:
ndcg_at_k(recommended_list, bought_list, 5)

0.48522855511632257